In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

## Wildstyle failures

In [2]:
ws = analyze.Analysis('results/wildstyle')
print(ws)

Analysis for Celeritas v0.3.2-rc.1 on wildstyle


In [3]:
ftab = analyze.make_failure_table(ws.failures())
ftab.to_frame()

,Failure


In [4]:
print("\n".join(k for (k, v) in ftab.items()
                if 'is_soft' in v))

In [5]:
summed = analyze.summarize_instances(ws.result[['unconverged']])

Unconverged tracks (from runs that didn't fail):

In [6]:
unconv = summed[('unconverged', 'mean')]
unconv[unconv > 0].unstack('arch') 

,arch
problem,geo


## Summit results

In [2]:
summit = analyze.Analysis('results/summit')
print(summit)
summed = analyze.summarize_instances(summit.result[summit.successful].dropna(how='all'))

Analysis for Celeritas v0.3.2-rc.1 on summit


In [3]:
deets = summit.load_results(('testem3-flat','orange','gpu'), 0)
deets['system']['device']

{'capability_major': 7,
 'capability_minor': 0,
 'clock_rate': 1530000,
 'default_block_size': 256,
 'device_id': 0,
 'eu_per_cu': 1,
 'max_blocks_per_grid': 2147483647,
 'max_blocks_per_multiprocessor': 32,
 'max_cache_size': 6291456,
 'max_threads_per_block': 1024,
 'max_threads_per_cu': 2048,
 'memory_clock_rate': 877000,
 'multiprocessor_count': 80,
 'name': 'Tesla V100-SXM2-16GB',
 'platform': 'cuda',
 'regs_per_block': 65536,
 'regs_per_multiprocessor': 65536,
 'shared_mem_per_block': 49152,
 'threads_per_warp': 32,
 'total_const_mem': 65536,
 'total_global_mem': 16911433728}

### Failures

Average number of unconverged tracks:

In [9]:
unconv = analyze.summarize_instances(summit.result['unconverged'])['mean']
unconv[unconv > 0].unstack('arch') 

,arch
problem,geo


### Timing tables

In [10]:
(fig, [run_ax, setup_ax]) = plt.subplots(nrows=2,
                                         gridspec_kw=dict(height_ratios=[3, 1]),
                                         subplot_kw=dict(yscale='log'))
summit.plot_results(run_ax, summed['total_time'])
run_ax.legend();
run_ax.set_ylabel('Run [s]')
run_ax.tick_params(labelbottom=False)
summit.plot_results(setup_ax, summed['setup_time'])
setup_ax.set_ylabel('Setup [s]')
analyze.annotate_metadata(run_ax, summit)
plt.tight_layout()
fig.savefig('plots/timing.pdf', transparent=True)
plt.close()

In [11]:
times = summed[('total_time', 'mean')].unstack()
times.style.format(float_fmt_transform(2))

In [12]:
event_rate = analyze.calc_event_rate(summit, summed)
event_rate['mean'].xs('simple-cms+field+msc', level='problem').unstack('arch')

arch,cpu,gpu
geo,,
orange,0.067483,1.921134
vecgeom,0.068711,1.969372


In [53]:
speedup = analyze.get_cpugpu_ratio(summed['total_time'])
speedup.dropna().style.format(analyze.float_fmt_transform(1))

In [112]:
#df = speedup.apply(np.vectorize(analyze.float_fmt_transform(1)))
speedup_out = np.full((len(speedup), 3), "", dtype=object)
_abbrev = summit.problem_to_abbr()
prev_prob = None
for (i, ((prob, geo), row)) in enumerate(speedup.iterrows()):
    if prob != prev_prob:
        abbr = _abbrev[prob]
        speedup_out[i, 0] = f"{prob} [{abbr}]"
    speedup_out[i, 1] = geo
    speedup_out[i, 2] = "{:.1f}× (±{:.1f})".format(*row)
    prev_prob = prob
    
headers = ["Problem", "Geometry", "Speedup"]
widths = np.vectorize(len)(np.concatenate([speedup_out, [headers]], axis=0))
col_widths = np.max(widths, axis=0)
fmt = f"| {{:<{col_widths[0]}}} | {{:<{col_widths[1]}}} | {{:>{col_widths[2]}}} |\n".format

with open("results/summit/speedup.md", "w") as f:
    f.write(fmt("Problem", "Geometry", "Speedup"))
    f.write(fmt(*["-"*w for w in col_widths]))
    for i in range(speedup_out.shape[0]):
        f.write(fmt(*speedup_out[i,:].tolist()))

In [107]:
_desc = (speedup['mean'].dropna() * 7).describe()
print("CPU:GPU equivalence: {min:.0f}× to {max:.0f}×".format(**_desc))

CPU:GPU equivalence: 46× to 240×


In [15]:
# Determine the fraction of action time spent in geometry routines
action_times_inst = analyze.unstack_subdict(summit.result['action_times'][summit.valid]).T
total_time_inst = summit.result['total_time']
geo_actions = [lab for lab in action_times_inst.index
               if lab.startswith('along-step-') or lab.startswith('geo-')]
geo_frac_inst = action_times_inst.loc[geo_actions].sum() / total_time_inst
geo_frac = analyze.summarize_instances(geo_frac_inst)

In [16]:
geo_frac['mean'].unstack('arch')

arch                                 cpu       gpu
problem                geo                        
cms2018                vecgeom  0.456218  0.759273
cms2018+field+msc      vecgeom  0.691245  0.898051
simple-cms+field       orange   0.329535  0.268992
simple-cms+field+msc   orange   0.498907  0.357718
                       vecgeom  0.492256  0.404019
simple-cms+msc         orange   0.429770  0.284143
testem15               orange   0.197602  0.000000
testem15+field         orange   0.313485  0.000000
testem15+field+msc     orange   0.463364  0.000000
                       vecgeom  0.447223  0.000000
testem3-flat           orange   0.270162  0.000000
                       vecgeom  0.301729  0.000000
testem3-flat+field     orange   0.380768  0.000000
testem3-flat+field+msc orange   0.595612  0.000000
                       vecgeom  0.653365  0.000000
testem3-flat+msc       orange   0.541909  0.000000

### Plots

In [17]:
problems = summit.problems()
problem_to_abbr = summit.problem_to_abbr(problems)
p_to_i = dict(zip(problems, itertools.count()))

In [103]:
fig, ax = plt.subplots()
summit.plot_results(ax, speedup)
ax.set_ylabel("Speedup (7-CPU / 1-GPU wall time)")
ax.set_ylim([0, None])
analyze.annotate_metadata(ax, summit)
plt.tight_layout()
fig.savefig('plots/speedups.pdf', transparent=True)
fig.savefig('results/summit/speedup.png', transparent=False, dpi=150)
plt.close()

In [19]:
fig, axes = plt.subplots(nrows=2, figsize=(4,4), subplot_kw=dict(yscale='log'))
for (ax, q) in zip(axes, ['step', 'primary']):
    summit.plot_results(ax, analyze.inverse_summary(summed['avg_time_per_' + q]))
    ax.set_ylabel(q + ' per sec')
    if ax != axes[-1]:
        ax.tick_params(labelbottom=False)
    ax.legend()
plt.tight_layout()
fig.savefig('plots/steps-vs-primaries.png', dpi=300)
plt.close()

In [20]:
(fig, (time_ax, geo_ax)) = plt.subplots(
    nrows=2, figsize=(4, 4),
    gridspec_kw=dict(height_ratios=[3, 1])
)
time_ax.set_yscale('log')
summit.plot_results(time_ax, event_rate)
time_ax.set_ylabel(r"Throughput [event/s]")
time_ax.set_ylim([0.5 * event_rate['mean'].min(), None])
time_ax.legend()
time_ax.tick_params(labelbottom=False)
analyze.annotate_metadata(time_ax, summit)
summit.plot_results(geo_ax, geo_frac * 100)
geo_ax.set_ylabel("Geometry [%]")
geo_ax.set_ylim([0, 100])
plt.tight_layout()
fig.savefig('plots/throughput-geo.pdf', transparent=True)
plt.close()

## Action fraction pie charts

In [5]:
avg_time = summed[('total_time', 'mean')].T
mean_action_times = summit.action_times().xs('mean', axis=1, level=1).T

In [6]:
mean_action_times.divide(avg_time, axis=1).dropna(how='all', axis=1).style.format(analyze.float_fmt_transform(2))

In [30]:
from enum import IntEnum
KernelCategory = IntEnum("KernelCategory", ["INIT",  "GEO", "PHYSMOD", "PHYSCORE",],
                        start=0)
category_labels = ["Initialization", "Geometry", "Physics models", "Core physics"]

KERNEL_ORDERING = {
    # along-step: GEO
    "geo-boundary": (KernelCategory.GEO, "zzz"),
    "extend-from-secondaries": (KernelCategory.INIT, "ex"),
    "initialize-tracks": (KernelCategory.INIT, "init"),
    "pre-step": (KernelCategory.PHYSCORE, ""),
    "physics-discrete-select": (KernelCategory.PHYSCORE, "b"),
    # other physics: 3
}
def get_action_priority(k):
    if k.startswith('along-step'):
        return (KernelCategory.GEO, k)
    try:
        return KERNEL_ORDERING[k]
    except KeyError:
        return (KernelCategory.PHYSMOD, k)
    
def autopct_format(pctvalue):
    if pctvalue < 5:
        return ""
    return "{:1.0f}%".format(pctvalue)


class PiePlotter:
    def __init__(self, times):
        self.times = times.dropna()

        actions = list(self.times.index)
        cpa = sorted([get_action_priority(a) + (a,) for a in actions])
        (self.category, _, self.actions) = zip(*cpa)
        self.labels = [a if c != KernelCategory.PHYSMOD else None
                      for (c, p, a) in cpa]
        
        cmap = plt.colormaps["tab20c"] # 5 groups of 4 shades
        def _get_color(color, shade = 0):
            shade = shade % 4
            return cmap((color % 5) * 4 + shade)
        
        self.outer_colors = _get_color(np.arange(len(KernelCategory)))
        self.inner_colors = _get_color(np.array(self.category),
                                       np.arange(len(self.category)))
        
        cat = np.array([int(c) for c in self.category])
        self.catbound = np.concatenate([cat[:-1] != cat[1:], [True]])
        self.catlabels = [category_labels[c] for c in cat[self.catbound]]

    def __call__(self, ax, arch):
        width = 0.3
        angle = 90.0 # degrees
        
        series = self.times[arch]
        inner = [series[t] for t in self.actions]
        outer = np.cumsum(inner)[self.catbound]
        outer = np.concatenate([[outer[0]], np.diff(outer)])
        
        (wedges, texts, autotexts) = ax.pie(
            outer,
            autopct=autopct_format, pctdistance=0.85,
            radius=1, colors=self.outer_colors,
            wedgeprops=dict(width=width, edgecolor='w'), startangle=angle,
        )
        outer_legend = ax.legend(wedges, self.catlabels,
                  loc="upper right",
                  bbox_to_anchor=(0.5, 0, 0.5, 1))
        
        (wedges, texts) = ax.pie(
            inner,
            radius=(1 - width), colors=self.inner_colors,
            wedgeprops=dict(width=width, edgecolor='w'), startangle=angle,
        )
        (wedges, labels) = zip(*[(w, l) for (w, l) in zip(wedges, self.labels) if l])
        inner_legend = ax.legend(wedges, labels,
                           loc="center",
                           fontsize='xx-small')
        ax.add_artist(outer_legend)

In [32]:
for prob in ["cms2018", "cms2018+field+msc"]:
    geo = 'vecgeom'
    if prob not in mean_action_times.columns.get_level_values('problem'):
        print("Missing problem:", prob)
        continue
#    plot_action_pie(summit.load_results((prob,'vecgeom','gpu'), 0))
    md = {k: getattr(summit, k) for k in ["version", "system"]}
    pieplot = PiePlotter(mean_action_times.xs((prob, geo), axis=1, level=('problem', 'geo')))
    
    # Loop over CPU/GPU
    for arch in pieplot.times:
        (fig, ax) = plt.subplots(figsize=(4, 4), subplot_kw=dict(aspect="equal"),
                                 layout="constrained")
        pieplot(ax, arch)
        name = (prob, geo, arch)
        slashname = "/".join(name)
        fig.text(
            0.98, 0.1, f"{slashname}\n{md['version']} on {md['system']}",
            va='bottom', ha='right',
            fontstyle='italic', color=(0.5,)*3, size='xx-small',
            zorder=-100
        )

        dashname = "-".join(name)        
        fig.savefig(f'plots/actions-{dashname}.png', transparent=True)
        plt.close()

In [27]:
!open plots/

In [25]:
times

arch,cpu,gpu
action_times,,
along-step-neutral,27.994210,5.694325
annihil-2-gamma,0.575923,0.084467
brems-rel,0.521747,0.086050
brems-sb,4.999211,0.221403
conv-bethe-heitler,1.094768,0.126126
extend-from-primaries,0.009031,0.010067
extend-from-secondaries,7.880508,0.514083
geo-boundary,14.118672,2.420621
initialize-tracks,8.478343,0.339927


## Plot per-step timing on GPU

In [25]:
for p in ['cms2018', 'cms2018+field+msc']:
    data = summit.load_results((p, 'vecgeom', 'gpu'), 0)
    (fig, axes) = plt.subplots(nrows=2, figsize=(3, 4), sharex=True)
    for i, ax, plot in zip(itertools.count(),
                           axes,
                           [analyze.plot_counts, analyze.plot_accum_time_inv]):
        objs = plot(ax, data)
        analyze.annotate_metadata(ax, data['_metadata'])
        if i == 0:
            ax.set_xlabel(None)
    fig.savefig(f'plots/per-step-{p}.pdf', transparent=True)
    plt.tight_layout()
    plt.close()
    
    (fig, ax) = plt.subplots(figsize=(3, 2))
    analyze.plot_time_per_step(ax, data)
    plt.tight_layout()
    fig.savefig(f'plots/time-per-step-{p}.pdf', transparent=True)
    plt.close()

In [26]:
!open .

## Crusher

In [27]:
crusher = analyze.Analysis('results/crusher')
print(crusher)

Analysis for Celeritas v0.3.2-rc.1 on crusher


In [28]:
# VecGeom failures aren't really failures; just missing capability
#failures = crusher.failures().xs('orange', level='geo').fillna(1)
#failures.groupby(['problem', 'arch']).count().unstack()

In [29]:
csum = analyze.summarize_instances(crusher.result[crusher.successful].dropna(how='all'))

In [30]:
csum[('total_time', 'mean')].unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,69.481841,21.352300
simple-cms+field+msc,orange,86.764187,18.147253
simple-cms+msc,orange,132.179430,15.268424
testem15,orange,85.156364,46.524777
testem15+field,orange,103.600624,18.583460
testem15+field+msc,orange,108.385038,15.387350
testem3-flat,orange,144.221822,15.395753
testem3-flat+field,orange,156.642303,26.384439
testem3-flat+field+msc,orange,215.270396,31.032743


In [31]:
rel_err = csum.xs('std', axis=1, level=1) / csum.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

avg_time_per_primary  avg_time_per_step  \
problem                geo    arch                                            
simple-cms+field       orange cpu               0.275322           0.275406   
                              gpu               0.042716           0.042619   
simple-cms+field+msc   orange cpu               0.323856           0.323853   
                              gpu                    NaN                NaN   
simple-cms+msc         orange cpu               0.312771           0.312517   
                              gpu               0.029446           0.029620   
testem15               orange cpu               0.492642           0.492691   
                              gpu               0.057418           0.057406   
testem15+field         orange cpu               0.341683           0.341710   
                              gpu               0.052608           0.052572   
testem15+field+msc     orange cpu               0.369380           0.369405   
                              gpu               0.041043           0.041146   
testem3-flat           orange cpu               0.358876           0.358915   
                              gpu               0.028147           0.028085   
testem3-flat+field     orange cpu               0.463632           0.463758   
                              gpu                    NaN                NaN   
testem3-flat+field+msc orange cpu               0.308447           0.308507   
                              gpu                    NaN                NaN   
testem3-flat+msc       orange cpu               0.358870           0.358856   
                              gpu               0.024087           0.023985   

                                    num_step_iters  pre_emptying_time  \
problem                geo    arch                                      
simple-cms+field       orange cpu              NaN           0.058877   
                              gpu              NaN           0.369525   
simple-cms+field+msc   orange cpu              NaN           0.028052   
                              gpu              NaN           0.142175   
simple-cms+msc         orange cpu              NaN           0.479313   
                              gpu         0.023591                NaN   
testem15               orange cpu              NaN           0.664846   
                              gpu              NaN                NaN   
testem15+field         orange cpu              NaN           0.538967   
                              gpu              NaN                NaN   
testem15+field+msc     orange cpu              NaN           0.448470   
                              gpu              NaN                NaN   
testem3-flat           orange cpu              NaN           0.425242   
                              gpu              NaN                NaN   
testem3-flat+field     orange cpu              NaN           0.541261   
                              gpu              NaN           0.373685   
testem3-flat+field+msc orange cpu              NaN           0.309780   
                              gpu              NaN           0.031708   
testem3-flat+msc       orange cpu              NaN           0.318105   
                              gpu              NaN                NaN   

                                    setup_time  slot_occupancy  total_time  
problem                geo    arch                                          
simple-cms+field       orange cpu          NaN             NaN    0.275322  
                              gpu     0.028559             NaN    0.042716  
simple-cms+field+msc   orange cpu          NaN             NaN    0.323856  
                              gpu     0.067381             NaN         NaN  
simple-cms+msc         orange cpu          NaN             NaN    0.312771  
                              gpu     0.106972        0.023168    0.029446  
testem15               orange cpu          NaN             NaN    0.492642

In [32]:
analyze.get_cpugpu_ratio(csum['total_time'])

,,mean,std
problem,geo,,
simple-cms+field,orange,3.254068,0.906636
simple-cms+field+msc,orange,4.781120,1.550076
simple-cms+msc,orange,8.657045,2.719646
testem15,orange,1.830344,0.907808
testem15+field,orange,5.574883,1.927291
testem15+field+msc,orange,7.043775,2.617844
testem3-flat,orange,9.367637,3.372142
testem3-flat+field,orange,5.936920,2.752780
testem3-flat+field+msc,orange,6.936879,2.139982


In [33]:
crusher_times = csum['total_time']
crusher_times

count        mean        std
problem                geo    arch                              
simple-cms+field       orange cpu     8.0   69.481841  19.129892
                              gpu     8.0   21.352300   0.912084
simple-cms+field+msc   orange cpu     8.0   86.764187  28.099084
                              gpu     8.0   18.147253   0.274085
simple-cms+msc         orange cpu     8.0  132.179430  41.341899
                              gpu     8.0   15.268424   0.449586
testem15               orange cpu     8.0   85.156364  41.951578
                              gpu     8.0   46.524777   2.671343
testem15+field         orange cpu     8.0  103.600624  35.398604
                              gpu     8.0   18.583460   0.977644
testem15+field+msc     orange cpu     8.0  108.385038  40.035300
                              gpu     8.0   15.387350   0.631540
testem3-flat           orange cpu     8.0  144.221822  51.757711
                              gpu     8.0   15.395753   0.433350
testem3-flat+field     orange cpu     8.0  156.642303  72.624419
                              gpu     8.0   26.384439   0.158990
testem3-flat+field+msc orange cpu     8.0  215.270396  66.399608
                              gpu     8.0   31.032743   0.165254
testem3-flat+msc       orange cpu     8.0  171.683129  61.611944
                              gpu     8.0   17.149040   0.413065

In [34]:
crusher_rates = analyze.calc_event_rate(crusher, csum)
summit_rates = analyze.calc_event_rate(summit, summed.loc[crusher_times.index])

counts = {
    ('summit', 'cpu'): 7,
    ('summit', 'gpu'): 1,
    ('crusher', 'cpu'): 8,
    ('crusher', 'gpu'): 1,
}

In [35]:
(crusher_rates['mean'] / summit_rates['mean']).unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,1.113305,0.149299
simple-cms+field+msc,orange,1.195534,0.200784
simple-cms+msc,orange,0.692312,0.218815
testem15,orange,0.724107,0.062677
testem15+field,orange,0.695997,0.146652
testem15+field+msc,orange,0.855457,0.187407
testem3-flat,orange,0.732642,0.271122
testem3-flat+field,orange,0.788517,0.205330
testem3-flat+field+msc,orange,1.004549,0.250785


In [36]:
fig, ax = plt.subplots()
ax.set_yscale('log')
for offset, color, machine, rates in [(-0.05, '#7A954F', 'Summit', summit_rates),
                                      (0.05, '#BC5544', 'Crusher', crusher_rates)]:
    for arch in ['cpu', 'gpu']:
        summary = rates.xs(arch, level='arch')
        index = np.array([p_to_i[p]
                          for p in summary.index.get_level_values('problem')], dtype=float)
        index += offset
    
        mark = analyze.ARCH_SHAPES[arch]
        count = counts[(machine.lower(), arch)]
        arch = arch.upper()
        ax.errorbar(index, summary['mean'], summary['std'],
                    capsize=0, fmt='none', ecolor=(0.2,)*3)
        scat = ax.scatter(index, summary['mean'], c=color, marker=mark,
                         label=f"{machine} ({count} {arch})")    
xax = ax.get_xaxis()
xax.set_ticks(np.arange(len(problems)))
xax.set_ticklabels(list(problem_to_abbr.values()), rotation=90)
grid = ax.grid()
ax.set_axisbelow(True)
ax.legend()
ax.set_ylabel(r"Event rate [1/s]")
analyze.annotate_metadata(ax, summit)
plt.tight_layout()
fig.savefig('plots/crusher-vs-summit.pdf')
plt.close()